In [1408]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

In [1409]:
df_Academia = pd.read_csv('./Datasets/users_data.csv')

## ANALISE EXPLORATORIA DOS DADOS

In [1410]:
df_Academia.head()

,user_id,first_name,last_name,age,gender,birthdate,sign_up_date,user_location,subscription_plan
0,user_1,Chris,Wilson,56,Female,2000-02-29,2023-02-06,Denver,Basic
1,user_2,Michael,Miller,46,Non-binary,1978-12-22,2023-08-08,Orlando,Pro
2,user_3,Daniel,Smith,32,Female,1962-08-30,2021-01-11,Orlando,Basic
3,user_4,David,Smith,60,Male,2003-12-05,2023-08-07,Denver,Pro
4,user_5,Chris,Jones,25,Female,2004-08-25,2021-01-08,Denver,Basic


In [1411]:
df_Academia.tail()

,user_id,first_name,last_name,age,gender,birthdate,sign_up_date,user_location,subscription_plan
4995,user_4996,Sarah,Williams,33,Male,1958-01-13,2021-08-08,Las Vegas,Basic
4996,user_4997,Linda,Brown,22,Male,1974-02-20,2023-01-22,Austin,Student
4997,user_4998,Emily,Miller,38,Non-binary,1978-03-29,2021-04-16,Atlanta,Pro
4998,user_4999,Robert,Garcia,27,Male,2002-02-01,2022-07-02,Denver,Pro
4999,user_5000,Chris,Smith,36,Male,1959-11-21,2022-02-20,Seattle,Student


In [1412]:
df_Academia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   user_id            5000 non-null   object
 1   first_name         5000 non-null   object
 2   last_name          5000 non-null   object
 3   age                5000 non-null   int64 
 4   gender             5000 non-null   object
 5   birthdate          5000 non-null   object
 6   sign_up_date       5000 non-null   object
 7   user_location      5000 non-null   object
 8   subscription_plan  5000 non-null   object
dtypes: int64(1), object(8)
memory usage: 351.7+ KB


In [1413]:
df_Academia.describe()

,age
count,5000.00000
mean,41.16820
std,13.53105
min,18.00000
25%,29.00000
50%,41.00000
75%,53.00000
max,64.00000


In [1414]:
df_Academia.head()

,user_id,first_name,last_name,age,gender,birthdate,sign_up_date,user_location,subscription_plan
0,user_1,Chris,Wilson,56,Female,2000-02-29,2023-02-06,Denver,Basic
1,user_2,Michael,Miller,46,Non-binary,1978-12-22,2023-08-08,Orlando,Pro
2,user_3,Daniel,Smith,32,Female,1962-08-30,2021-01-11,Orlando,Basic
3,user_4,David,Smith,60,Male,2003-12-05,2023-08-07,Denver,Pro
4,user_5,Chris,Jones,25,Female,2004-08-25,2021-01-08,Denver,Basic


alguns problemas encontrados com os dados:

    Generos incorretos
    Data de nascimento e idade nao batem
    user_id pode ser alterado, identificando usuarios apenas por numeros

encontradas tambem informações desnecessarias para os objetivos do estudo:

    localização
    nome e sobrenome
    data de nascimento

## TRATAMENTO DE DADOS

In [1415]:
df_Academia.head()

,user_id,first_name,last_name,age,gender,birthdate,sign_up_date,user_location,subscription_plan
0,user_1,Chris,Wilson,56,Female,2000-02-29,2023-02-06,Denver,Basic
1,user_2,Michael,Miller,46,Non-binary,1978-12-22,2023-08-08,Orlando,Pro
2,user_3,Daniel,Smith,32,Female,1962-08-30,2021-01-11,Orlando,Basic
3,user_4,David,Smith,60,Male,2003-12-05,2023-08-07,Denver,Pro
4,user_5,Chris,Jones,25,Female,2004-08-25,2021-01-08,Denver,Basic


In [1416]:
df_Academia.first_name.unique()

array(['Chris', 'Michael', 'Daniel', 'David', 'Jessica', 'Linda', 'Jane',
       'Robert', 'John', 'Sarah', 'Emily', 'Laura'], dtype=object)

In [1417]:
#Definindo função para definir genero correto aos clientes
nomes_masculinos = ['Michael','Daniel','David','Robert','John']

def atualizaGenero(row):
    if (row['first_name'] in nomes_masculinos):
        return 'Male'
    if ((row['first_name'] == 'Chris')& (row['gender'] != 'Non-binary')):
        return row['gender']
    else:
        return 'Female'

In [1418]:
df_Academia['gender'] = df_Academia.apply(atualizaGenero,axis=1)

In [1419]:
df_Academia.head(30)

,user_id,first_name,last_name,age,gender,birthdate,sign_up_date,user_location,subscription_plan
0,user_1,Chris,Wilson,56,Female,2000-02-29,2023-02-06,Denver,Basic
1,user_2,Michael,Miller,46,Male,1978-12-22,2023-08-08,Orlando,Pro
2,user_3,Daniel,Smith,32,Male,1962-08-30,2021-01-11,Orlando,Basic
3,user_4,David,Smith,60,Male,2003-12-05,2023-08-07,Denver,Pro
4,user_5,Chris,Jones,25,Female,2004-08-25,2021-01-08,Denver,Basic
5,user_6,Jessica,Rodriguez,38,Female,1969-09-11,2022-02-13,Austin,Student
6,user_7,Linda,Jones,56,Female,1982-07-09,2023-07-25,Seattle,Student
7,user_8,Jane,Brown,36,Female,1989-05-17,2021-02-01,Atlanta,Pro
8,user_9,Robert,Davis,40,Male,1976-04-10,2023-09-11,Orlando,Student
9,user_10,John,Jones,28,Male,2005-10-30,2022-06-24,Detroit,Basic


In [1420]:
#Transformando datas em datetime
df_Academia['birthdate'] = pd.to_datetime(df_Academia['birthdate'])
df_Academia['sign_up_date'] = pd.to_datetime(df_Academia['sign_up_date'])

In [1421]:
#Corrigindo a idade dos clientes
df_Academia['age'] = ((df_Academia['sign_up_date'] - df_Academia['birthdate']).dt.days /365.25).astype(int)

In [1422]:
df_Academia.head()

,user_id,first_name,last_name,age,gender,birthdate,sign_up_date,user_location,subscription_plan
0,user_1,Chris,Wilson,22,Female,2000-02-29,2023-02-06,Denver,Basic
1,user_2,Michael,Miller,44,Male,1978-12-22,2023-08-08,Orlando,Pro
2,user_3,Daniel,Smith,58,Male,1962-08-30,2021-01-11,Orlando,Basic
3,user_4,David,Smith,19,Male,2003-12-05,2023-08-07,Denver,Pro
4,user_5,Chris,Jones,16,Female,2004-08-25,2021-01-08,Denver,Basic


In [1423]:
df_Academia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   user_id            5000 non-null   object        
 1   first_name         5000 non-null   object        
 2   last_name          5000 non-null   object        
 3   age                5000 non-null   int64         
 4   gender             5000 non-null   object        
 5   birthdate          5000 non-null   datetime64[ns]
 6   sign_up_date       5000 non-null   datetime64[ns]
 7   user_location      5000 non-null   object        
 8   subscription_plan  5000 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 351.7+ KB


In [1424]:
#Retirando informação de mes e ano de ingresso do cliente
df_Academia['Mes'] = df_Academia['sign_up_date'].dt.month
df_Academia['Ano'] = df_Academia['sign_up_date'].dt.year

In [1425]:
df_Academia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   user_id            5000 non-null   object        
 1   first_name         5000 non-null   object        
 2   last_name          5000 non-null   object        
 3   age                5000 non-null   int64         
 4   gender             5000 non-null   object        
 5   birthdate          5000 non-null   datetime64[ns]
 6   sign_up_date       5000 non-null   datetime64[ns]
 7   user_location      5000 non-null   object        
 8   subscription_plan  5000 non-null   object        
 9   Mes                5000 non-null   int32         
 10  Ano                5000 non-null   int32         
dtypes: datetime64[ns](2), int32(2), int64(1), object(6)
memory usage: 390.8+ KB


In [1426]:
#retirando colunas que nao serao utilizadas
df_Academia.drop(columns=['first_name','last_name','birthdate','sign_up_date', 'user_location'], inplace=True)

In [1427]:
#Definindo função para alterar de tipo de plano para custo do mesmo
valoresPlanos = [19.99, 49.99, 9.99]
def atualizaPlanos(row):
    if row['subscription_plan'] == 'Basic':
        return valoresPlanos[0]
    if row['subscription_plan'] == 'Pro':
        return valoresPlanos[1]
    if row['subscription_plan'] == 'Student':
        return valoresPlanos[2]

In [1428]:
df_Academia.subscription_plan = df_Academia.apply(atualizaPlanos, axis=1)

In [1429]:
df_Academia.head()

,user_id,age,gender,subscription_plan,Mes,Ano
0,user_1,22,Female,19.99,2,2023
1,user_2,44,Male,49.99,8,2023
2,user_3,58,Male,19.99,1,2021
3,user_4,19,Male,49.99,8,2023
4,user_5,16,Female,19.99,1,2021


In [1430]:
#Renomeando colunas para portugues
df_Academia.rename(columns={'user_id':'IdUsuario','age':'Idade','gender':'genero','subscription_plan':'ValorPlano','Mes':'MesEntrada','Ano':'AnoEntrada'}, inplace=True)

In [1431]:
df_Academia.head()

,IdUsuario,Idade,genero,ValorPlano,MesEntrada,AnoEntrada
0,user_1,22,Female,19.99,2,2023
1,user_2,44,Male,49.99,8,2023
2,user_3,58,Male,19.99,1,2021
3,user_4,19,Male,49.99,8,2023
4,user_5,16,Female,19.99,1,2021


In [1432]:
#definindo estação em que cliente ingressou
bins = [0,3,6,9,12]
labels = ['Primavera','Verao','Outono','Inverno']
df_Academia['Estacao'] = pd.cut(df_Academia.MesEntrada, bins=bins, labels=labels)

In [1433]:
#Retirando coluna que nao sera usada
df_Academia.drop(columns=['IdUsuario'], inplace=True)

In [1434]:
df_Academia.head()

,Idade,genero,ValorPlano,MesEntrada,AnoEntrada,Estacao
0,22,Female,19.99,2,2023,Primavera
1,44,Male,49.99,8,2023,Outono
2,58,Male,19.99,1,2021,Primavera
3,19,Male,49.99,8,2023,Outono
4,16,Female,19.99,1,2021,Primavera


In [1435]:
#verificando a existencia de nulos
df_Academia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Idade       5000 non-null   int64   
 1   genero      5000 non-null   object  
 2   ValorPlano  5000 non-null   float64 
 3   MesEntrada  5000 non-null   int32   
 4   AnoEntrada  5000 non-null   int32   
 5   Estacao     5000 non-null   category
dtypes: category(1), float64(1), int32(2), int64(1), object(1)
memory usage: 161.5+ KB


In [1436]:
#transformando valor do plano de dolar para real
df_Academia.ValorPlano = (df_Academia.ValorPlano * 5.75).astype(int)

In [1437]:
df_Academia.head()

,Idade,genero,ValorPlano,MesEntrada,AnoEntrada,Estacao
0,22,Female,114,2,2023,Primavera
1,44,Male,287,8,2023,Outono
2,58,Male,114,1,2021,Primavera
3,19,Male,287,8,2023,Outono
4,16,Female,114,1,2021,Primavera


In [1438]:
#usando o one hot encoding para gerar colunas normalizadas para genero e estação
df_Academia = pd.get_dummies(df_Academia, columns=['genero','Estacao']).astype(int)

In [1439]:
df_Academia['TotalClientes'] = 1

In [1440]:
df_Academia.head()

,Idade,ValorPlano,MesEntrada,AnoEntrada,genero_Female,genero_Male,Estacao_Primavera,Estacao_Verao,Estacao_Outono,Estacao_Inverno,TotalClientes
0,22,114,2,2023,1,0,1,0,0,0,1
1,44,287,8,2023,0,1,0,0,1,0,1
2,58,114,1,2021,0,1,1,0,0,0,1
3,19,287,8,2023,0,1,0,0,1,0,1
4,16,114,1,2021,1,0,1,0,0,0,1


In [1441]:
df_Academia.head(1)

,Idade,ValorPlano,MesEntrada,AnoEntrada,genero_Female,genero_Male,Estacao_Primavera,Estacao_Verao,Estacao_Outono,Estacao_Inverno,TotalClientes
0,22,114,2,2023,1,0,1,0,0,0,1


In [1442]:
df_Academia = df_Academia.groupby(['MesEntrada','AnoEntrada','ValorPlano','Estacao_Verao','Estacao_Outono','Estacao_Inverno','Estacao_Primavera']).agg({
    'genero_Male':'sum',
    'genero_Female':'sum',
    'TotalClientes':'sum'

})

In [1443]:
df_Academia.head()

genero_Male  \
MesEntrada AnoEntrada ValorPlano Estacao_Verao Estacao_Outono Estacao_Inverno Estacao_Primavera                
1          2021       57         0             0              0               1                           27   
                      114        0             0              0               1                           28   
                      287        0             0              0               1                           30   
           2022       57         0             0              0               1                           26   
                      114        0             0              0               1                           26   

                                                                                                 genero_Female  \
MesEntrada AnoEntrada ValorPlano Estacao_Verao Estacao_Outono Estacao_Inverno Estacao_Primavera                  
1          2021       57         0             0              0               1                             29   
                      114        0             0              0               1                             30   
                      287        0             0              0               1                             29   
           2022       57         0             0              0               1                             32   
                      114        0             0              0               1                             18   

                                                                                                 TotalClientes  
MesEntrada AnoEntrada ValorPlano Estacao_Verao Estacao_Outono Estacao_Inverno Estacao_Primavera                 
1          2021       57         0             0              0               1                             56  
                      114        0             0              0               1                             58  
                      287        0             0              0               1                             59  
           2022       57         0             0              0               1                             58  
                      114        0             0              0               1                             44

Dados prontos para aplicação de modelo

## TREINAMENTO DE MODELO

In [1444]:
df_Academia.head(5)

genero_Male  \
MesEntrada AnoEntrada ValorPlano Estacao_Verao Estacao_Outono Estacao_Inverno Estacao_Primavera                
1          2021       57         0             0              0               1                           27   
                      114        0             0              0               1                           28   
                      287        0             0              0               1                           30   
           2022       57         0             0              0               1                           26   
                      114        0             0              0               1                           26   

                                                                                                 genero_Female  \
MesEntrada AnoEntrada ValorPlano Estacao_Verao Estacao_Outono Estacao_Inverno Estacao_Primavera                  
1          2021       57         0             0              0               1                             29   
                      114        0             0              0               1                             30   
                      287        0             0              0               1                             29   
           2022       57         0             0              0               1                             32   
                      114        0             0              0               1                             18   

                                                                                                 TotalClientes  
MesEntrada AnoEntrada ValorPlano Estacao_Verao Estacao_Outono Estacao_Inverno Estacao_Primavera                 
1          2021       57         0             0              0               1                             56  
                      114        0             0              0               1                             58  
                      287        0             0              0               1                             59  
           2022       57         0             0              0               1                             58  
                      114        0             0              0               1                             44

In [1445]:
X = df_Academia.drop(columns=['TotalClientes','genero_Male','genero_Female'])
y= df_Academia['TotalClientes']

In [1446]:
X_test, X_train, y_test, y_train =train_test_split(X,y,train_size=0.70, random_state=10)

In [1447]:
X_train.head()

Empty DataFrame
Columns: []
Index: [(9, 2023, 114, 0, 1, 0, 0), (8, 2021, 57, 0, 1, 0, 0), (10, 2022, 114, 0, 0, 1, 0), (5, 2022, 57, 1, 0, 0, 0), (5, 2021, 287, 1, 0, 0, 0)]

In [1448]:
X_train.shape

(31, 0)

In [1449]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 31 entries, (9, 2023, 114, 0, 1, 0, 0) to (11, 2022, 114, 0, 0, 1, 0)
Empty DataFrame


In [1450]:
print(X_train.columns)

Index([], dtype='object')


In [1451]:
X_train = X_train.reset_index()
X_train.head()

,MesEntrada,AnoEntrada,ValorPlano,Estacao_Verao,Estacao_Outono,Estacao_Inverno,Estacao_Primavera
0,9,2023,114,0,1,0,0
1,8,2021,57,0,1,0,0
2,10,2022,114,0,0,1,0
3,5,2022,57,1,0,0,0
4,5,2021,287,1,0,0,0


In [1452]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   MesEntrada         31 non-null     int64
 1   AnoEntrada         31 non-null     int64
 2   ValorPlano         31 non-null     int64
 3   Estacao_Verao      31 non-null     int64
 4   Estacao_Outono     31 non-null     int64
 5   Estacao_Inverno    31 non-null     int64
 6   Estacao_Primavera  31 non-null     int64
dtypes: int64(7)
memory usage: 1.8 KB


In [1453]:
y_train.info()

<class 'pandas.core.series.Series'>
MultiIndex: 31 entries, (9, 2023, 114, 0, 1, 0, 0) to (11, 2022, 114, 0, 0, 1, 0)
Series name: TotalClientes
Non-Null Count  Dtype
--------------  -----
31 non-null     int64
dtypes: int64(1)
memory usage: 1.1 KB


In [1454]:
model_1 = LinearRegression().fit(X_train, y_train)

In [1455]:
X_test = X_test.reset_index()

In [1456]:
y_pred = model_1.predict(X_test)

In [1457]:
y_pred

array([52.51913403, 51.24832806, 45.89633144, 53.26470888, 54.56934438,
       50.62485539, 49.21734776, 45.91556149, 54.54755126, 52.49734091,
       47.18173688, 51.87643131, 46.5364711 , 50.79018157, 51.85720125,
       46.55826422, 49.25117729, 47.96577185, 50.48096009, 55.22561716,
       49.19811771, 55.19025398, 50.75738146, 51.89103078, 45.47201448,
       42.7463013 , 50.54758973, 51.24576499, 44.15380893, 51.4328843 ,
       51.21449853, 47.42176567, 47.17917382, 49.22938417, 49.26218428,
       53.95944177, 44.82931176, 54.58291444, 53.92664166, 48.55541498,
       52.56397055, 44.76268212, 54.60214449, 48.58924451, 49.87208689,
       51.88846772, 52.56653361, 46.53903416, 48.06446839, 49.92668012,
       48.60847457, 49.90745007, 48.62204462, 55.24484721, 52.49990397,
       53.94021171, 49.19555464, 52.53117044, 51.30035822, 51.26755811,
       54.58035137, 47.94654179, 53.14004363, 51.23372858, 48.13109803,
       48.70717111, 53.28393894, 50.60562534, 53.20667327, 44.79

In [1458]:
model_1.coef_

array([ -2.65908355,   0.64270272,  -0.01185093,  -3.18605447,
         4.7719661 ,   8.95648388, -10.54239551])

In [1459]:
model_1.intercept_

-1230.4448693872566

In [1460]:
#Score Conjunto de treinamento
model_1.score(X_train, y_train)

0.1441183284980716

In [1461]:
#Score conjunto de testes
model_1.score(X_test,y_test)

-0.15921010467965457

In [1462]:
#R2 score
r2_score(y_test, y_pred)

-0.15921010467965457

In [ ]:
#erro medio absoluto
mean_absolute_error(y_test,y_pred)
#Modelo entregando 6% de erro medio absoluto

6.152221929405045

In [1464]:
len(X_test)

71

In [1465]:
len(y_test)

71

## PREDIÇÃO USANDO MODELO

In [1470]:
X_test.head(10)

,MesEntrada,AnoEntrada,ValorPlano,Estacao_Verao,Estacao_Outono,Estacao_Inverno,Estacao_Primavera
0,4,2023,287,1,0,0,0
1,8,2022,114,0,1,0,0
2,9,2021,287,0,1,0,0
3,7,2021,114,0,1,0,0
4,4,2023,114,1,0,0,0
5,5,2021,114,1,0,0,0
6,5,2022,287,1,0,0,0
7,6,2021,287,1,0,0,0
8,1,2022,114,0,0,0,1
9,1,2022,287,0,0,0,1


In [1472]:
#Previsão para ingresso de clientes na academia em abril de 2026 no plano de 114 reais
model_1.predict([[4,2026,287,1,0,0,0]])

/home/nicolas-giacomelli/.local/share/virtualenvs/Gym_DataMining-QMlOvIiO/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([54.4472422])

Considerando a taxa de erro de 6% podemos estimar que tem 3 clientes a mais ou a menos do que o valor correto